# Busca de Índice e Consulta de documento

**Autor:** Davi J. Leite Santos  
**Versão:** 0.0.3  
**Data:** 25 de Abril de 2024  
**Localização:** Ribeirão das Neves, Minas Gerais - Brasil  

## Contato
- 🏠 **Endereço:** Ribeirão das Neves, Minas Gerais - Brasil
- 📧 **Email:** davi.jls@outlook.com
- 🌐 **LinkedIn:** davi-j-leite-santos
- 🌐 **Website:** davijls.com.br

## Principais Competências
- **Cibersegurança**
- **Segurança da Informação**
- **Operações de TI**

In [5]:
import json
import re
import os
import time
import nltk
from nltk.stem import RSLPStemmer

In [6]:
# Baixar os recursos necessários do NLTK
nltk.download('rslp')
nltk.download('punkt')

# Inicializar o Stemmer para português
stemmer = RSLPStemmer()

[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\davim\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\davim\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Funções de criação de chunk para a analise de granularidade

In [7]:
# Função para carregar o índice e vocabulário
def load_data(index_file, vocab_file):
    with open(index_file, 'r', encoding='utf-8') as f:
        index = json.load(f)
    with open(vocab_file, 'r', encoding='utf-8') as f:
        vocab = json.load(f)
    return index, vocab


In [8]:
# Função para criar chunks de um documento
def create_chunks(text, chunk_size):
    words = text.split()
    return [' '.join(words[i:i + chunk_size]) for i in range(0, len(words), chunk_size)]


In [9]:
# Função para processar arquivo com granularidade ajustável
def process_file(file_path, chunk_size, doc_id, vocab, index):
    if not os.path.exists(file_path):
        print(f"Arquivo não encontrado: {file_path}")
        return
    with open(file_path, 'r', encoding='utf-8') as f:
        text = f.read()
    chunks = create_chunks(text, chunk_size)
    for chunk_id, chunk in enumerate(chunks):
        tokens = nltk.word_tokenize(chunk, language='portuguese')
        for pos, token in enumerate(tokens):
            stemmed_token = stemmer.stem(token)
            if stemmed_token not in vocab:
                vocab[stemmed_token] = len(vocab)
            word_id = vocab[stemmed_token]
            if word_id not in index:
                index[word_id] = {}
            if doc_id not in index[word_id]:
                index[word_id][doc_id] = []
            index[word_id][doc_id].append((chunk_id, pos))

In [10]:
# Função para indexar documentos com diferentes tamanhos de chunks
def index_documents_with_different_chunk_sizes(files_dir, chunk_sizes):
    results = []
    files = sorted(os.listdir(files_dir))
    
    for chunk_size in chunk_sizes:
        vocab = {}
        index = {}
        for doc_id, file_name in enumerate(files):
            file_path = os.path.join(files_dir, file_name)
            process_file(file_path, chunk_size, doc_id, vocab, index)
        
        # Salvar resultados temporariamente para medição
        temp_vocab_file = f'temp_vocab_{chunk_size}.json'
        temp_index_file = f'temp_index_{chunk_size}.json'
        with open(temp_vocab_file, 'w', encoding='utf-8') as f:
            json.dump(vocab, f)
        with open(temp_index_file, 'w', encoding='utf-8') as f:
            json.dump(index, f)
        
        # Medir o tamanho dos arquivos
        vocab_size = os.path.getsize(temp_vocab_file)
        index_size = os.path.getsize(temp_index_file)
        total_size = index_size + vocab_size
        
        # Adicionar resultados
        results.append({
            'chunk_size': chunk_size,
            'index_size': index_size,
            'vocab_size': vocab_size,
            'total_size': total_size,
            'index_file': temp_index_file,
            'vocab_file': temp_vocab_file,
        })
    
    return results

In [11]:
# Função para buscar termos no índice
def search_term(index, vocab, term):
    stemmed_term = stemmer.stem(term)
    if stemmed_term in vocab:
        word_id = vocab[stemmed_term]
        if str(word_id) in index:
            return index[str(word_id)]
    return {}

In [12]:
# Função para medir o tempo de pesquisa com diferentes tamanhos de chunks
def measure_search_time_with_different_chunk_sizes(index_file, vocab_file, search_terms):
    index, vocab = load_data(index_file, vocab_file)
    search_times = []
    for term in search_terms:
        start_time = time.time()
        search_term(index, vocab, term)
        end_time = time.time()
        search_times.append(end_time - start_time)
    avg_search_time = sum(search_times) / len(search_times)
    return avg_search_time

In [13]:
# Função para buscar termos no índice
def search_term(index, vocab, term):
    stemmed_term = stemmer.stem(term)
    if stemmed_term in vocab:
        word_id = vocab[stemmed_term]
        if str(word_id) in index:
            return index[str(word_id)]
    return {}


In [14]:
# Função para salvar o progresso em um arquivo
def save_progress(progress_file, progress):
    with open(progress_file, 'w', encoding='utf-8') as f:
        f.write("Progress:\n")
        for key, value in progress.items():
            f.write(f"{key}: {value}\n")

In [15]:
# Função principal para realizar a análise de hiperparâmetros
def analyze_hyperparameters(files_dir, search_terms, chunk_sizes, output_file, progress_file):
    progress = {}
    try:
        # Tenta carregar o progresso anterior
        with open(progress_file, 'r', encoding='utf-8') as f:
            for line in f:
                key, value = line.strip().split(": ")
                progress[key] = int(value)
    except FileNotFoundError:
        pass
    
    results = index_documents_with_different_chunk_sizes(files_dir, chunk_sizes)
    for i, result in enumerate(results):
        if 'current_index' not in progress:
            progress['current_index'] = i
        else:
            progress['current_index'] += 1
        save_progress(progress_file, progress)
        
        avg_search_time = measure_search_time_with_different_chunk_sizes(result['index_file'], result['vocab_file'], search_terms)
        result['avg_search_time'] = avg_search_time
        
        # Salvar resultados parciais em um arquivo
        with open(output_file, 'a', encoding='utf-8') as f:
            f.write(f"Chunk Size: {result['chunk_size']}\n")
            f.write(f"Index Size: {result['index_size']} bytes\n")
            f.write(f"Vocab Size: {result['vocab_size']} bytes\n")
            f.write(f"Total Size: {result['total_size']} bytes\n")
            f.write(f"Average Search Time: {result['avg_search_time']:.6f} segundos\n")
            f.write("\n")
    
    # Remover o arquivo de progresso após a conclusão
    os.remove(progress_file) if os.path.exists(progress_file) else None

In [16]:
# Definir os parâmetros
files_dir = '../Motor_de_busca-WebCrawler/sites_visitados'
search_terms = ['educação', 'linguagem', 'noticia', 'ideia', 'politica']
chunk_sizes = [50, 100, 200, 500]  # Tamanhos de chunks para testar
output_file = 'hyperparameter_analysis.txt'
progress_file = 'hyperparametro_progress.txt'

In [17]:
# Realizar a análise de hiperparâmetros
analyze_hyperparameters(files_dir, search_terms, chunk_sizes, output_file, progress_file)

Arquivo não encontrado: ../Motor_de_busca-WebCrawler/sites_visitados\blogbvps_wordpress_com]2019]11]22]a-educacao-pela-pratica-da-linguagem-uma-chance-pedagogico-filologica-na-poesia-de-joao-cabral-de-melo-neto-e-na-filosofia-de-paulo-freire-por-rafael-zacca-ufrj].txt
Arquivo não encontrado: ../Motor_de_busca-WebCrawler/sites_visitados\blogger_googleusercontent_com]img]b]R29vZ2xl]AVvXsEhR5ror77j37iSDDHw1FUaSHKnDQwXdDClY8bgoxAaMmPYyALTU6SPeOVqm5b08ifGtF1ZH-TTbBifE9aRN64iJfbTJC5UJ_BDeT_kABv0a9RalSvnvgY3XSXNOdBOePOeGIbk-82iODhU]s337]LOUCO.jpg.txt
Arquivo não encontrado: ../Motor_de_busca-WebCrawler/sites_visitados\capital_sp_gov_br]web]prefeitura-de-sao-paulo]w]prefeitura-oficializa-doa%C3%A7%C3%A3o-da-parque-princesa-isabel-ao-estado-para-viabilizar-a-transforma%C3%A7%C3%A3o-da-regi%C3%A3o-central%C2%A0.txt
Arquivo não encontrado: ../Motor_de_busca-WebCrawler/sites_visitados\epocanegocios_globo_com]podcast]negnews]noticia]2024]04]como-a-vivo-usa-ia-para-melhorar-a-experiencia-do-cliente-

Traceback (most recent call last):
  File "C:\Users\davim\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\davim\AppData\Local\Temp\ipykernel_25756\1679607890.py", line 2, in <module>
    analyze_hyperparameters(files_dir, search_terms, chunk_sizes, output_file, progress_file)
  File "C:\Users\davim\AppData\Local\Temp\ipykernel_25756\938647413.py", line 13, in analyze_hyperparameters
    results = index_documents_with_different_chunk_sizes(files_dir, chunk_sizes)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\davim\AppData\Local\Temp\ipykernel_25756\3009419626.py", line 11, in index_documents_with_different_chunk_sizes
    process_file(file_path, chunk_size, doc_id, vocab, index)
  File "C:\Users\davim\AppData\Local\Temp\ipykernel_25756\1797409972.py", line 12, in process_file
    stemmed_token = stemmer.stem(t